###  Решение СЛАУ методом вращений

In [37]:
import numpy as np
from math import cos, sin
from scipy import linalg

In [259]:
import numpy as np

def solve_with_rotation(m):
    """Solve system with rotation method.
    :param m: numpy matrix
    :return: None
    """
    n = m.shape[0]
    # прямой ход
    for i in range(n-1):
        for j in range(i + 1, n):
            c = m[i, i] / (m[i, i]**2 + m[j, i]**2) ** .5
            s = m[j, i] / (m[i, i]**2 + m[j, i]**2) ** .5
            tmp1 = m[i, :] * c + m[j, :] * s
            tmp2 = m[i, :] * -s + m[j, :] * c
            m[i, :] = tmp1
            m[j, :] = tmp2

    # проверка на особенность
    if is_singular(m):
        print('The system has infinite number of answers...')
        return

    # обратный ход
    answer = np.matrix([0.0 for i in range(n)]).T
    for k in range(n - 1, -1, -1):
        answer[k, 0] = (m[k, -1] - m[k, k:n] * answer[k:n, 0]) / m[k, k]

    # нахождение сз
    e_val = np.diag(m)
    return answer


def is_singular(m):
    """Check matrix for nonsingularity.
    :param m: matrix (list of lists)
    :return: True if system is nonsingular
    """
    return np.any(np.diag(m) == 0)



In [264]:
m = np.array([[63., 22., 0., 26., 1.], [22., 15.,1.,27., -2.], [0., 1., 21., 1., 0.], [26.,27., 1.,55., 4.]])

A = np.array([[63., 22., 0., 26.], [22., 15.,1.,27.], [0., 1., 21., 1.], [26.,27., 1.,55.]])
b = np.array([1,-2,0,4])
print(np.linalg.solve(A,b))
#b = np.array([1,-2,0,4])
answer = solve_with_rotation(m)
print(answer)

[ 14.97633136 -82.19970414   2.32840237  33.30325444]
[[ 14.97633136]
 [-82.19970414]
 [  2.32840237]
 [ 33.30325444]]


___

### Решение СЛАУ методом QR-разложения

In [187]:
def gram_schmidt_qr(A): # 5 pts
    n = A.shape[1] + 1
    Q = np.zeros((n, n))
    R = Q.copy()
    V = R.copy()
    V[1:,1:] = A

    for k in range(1, n):
        R[k][k] = np.linalg.norm(V[:,k], 2)
        Q[:,k] = V[:,k] / R[k][k]
        for i in range(k + 1, n):
            R[k][i] = np.dot(Q[:,k].T, V[:,i])
            V[:,i] = V[:,i] - R[k][i] * Q[:,k]

    return Q[1:, 1:], R[1:, 1:]

In [188]:
A = np.array([
        [3,7,1,2],
        [1, 2, 3, 1],
        [4, -2, 0, 1],
        [1,2,7,1]
])

m, n = A.shape
q,r = gram_schmidt_qr(A)
b = np.array([1,-2,0,4])
answer = linalg.inv(r) @ (q.T @ b)
print(answer)

[  5.34615385   2.38461538   1.5        -16.61538462]


### Вариационный метод

In [193]:
def variative(A, b, num_iter=100):
    x = b
    for k in range(num_iter):
        r = A @ x - b
        if np.array_equal(r, np.zeros(r.size)):
            break;
        t = (r @ A @ r) / (r @ A.T @ A @ r)
        x = x - t * r 
    return x

In [194]:
A = np.array([[63, 22, 0, 26], [22, 15,1,27], [0, 1, 21, 1], [26,27, 1,55]])
b = np.array([1,-2,0,4])

In [198]:
x = variative(A, b, 100000)
print(x)

[ 14.97633136 -82.19970414   2.32840237  33.30325444]


### Итерационный метод

In [218]:
def iterative(A, b, num_iter = 1000):
    x = b
    s,v = linalg.eig(A)
    coef = 2/(s.min()+s.max())
    for k in range(num_iter):
        r = A @ x - b
        if np.array_equal(r, np.zeros(r.size)):
            break;
        x = x - coef*r
    return x

In [219]:
A = np.array([[3, 1], [2, 6]])
b = np.array([5, 9])

In [220]:
x = iterative(A, b, 100)
print(x)

[1.3125+0.j 1.0625+0.j]


### Нахождение СЗ и СВ методом вращений

In [115]:
def rotation_method(m):
    n = m.shape[0]
    m_max = 1
    H = np.eye(n)
    einnumb = np.diag(m)
    
    while m_max > 0.1:
        a = m.copy()
        size = m.shape[0]
        for i in range(size):
            a[i][i] = 0 
        m_max = (abs(a)).max()
        m_max_ind = np.argmax(abs(a))
        i = m_max_ind // size
        j = m_max_ind % size
        p = (2*m[i][j])/(m[i][i]-m[j][j])
        s = sin(0.5* np.arctan(p))
        c = cos(0.5* np.arctan(p))
        H_temp = np.eye(n)
        H_temp[i][i] = c
        H_temp[j][j] = c
        H_temp[i][j] = -s
        H_temp[j][i] = s
        m = H_temp.T @ m @ H_temp
        H = H @ H_temp
        einnumb = np.diag(m)
        

    if is_singular(m):
        print('Матрица особенная')
        return

    return H, einnumb


def is_singular(m):
    return np.any(np.diag(m) == 0)



In [199]:
A = np.array([[63, 22, 0, 26], [22, 15,1,27], [0, 1, 21, 1], [26,27, 1,55]])
H, einnumb = rotation_method(A)
print(H)
print(einnumb)

[[ 0.71418495 -0.16662703  0.03349557  0.67900909]
 [-0.14709963  0.91378862  0.00922964  0.3785061 ]
 [-0.06520783 -0.02596098  0.99744909  0.0130109 ]
 [-0.68121179 -0.36952918 -0.06235524  0.62889626]]
[3.36684662e+01 3.85940735e-02 2.09467451e+01 9.93461946e+01]
